# アノテーション (jetson環境）アルファ版

PC環境だったものをJetson環境でも使えるようにしました。<br>
使用の際は、自己責任にてお願いいたします。<br>
Revision 0.0.0<br>
2024/10/22<br>
使用の際は、xy_dataset2.pyもjetsonへアップロードお願いいたします。
アノテーションでは、11_record_cameraで撮影した走行データにアノテーションを実施し、転移学習をおこないます。

### ボードの識別

走行データはcameraフォルダに録画されています。今度は、cameraフォルダのデータにアノテーションをおこないます。

In [1]:
# logウィジェット作成
from ipywidgets.widgets import Textarea
from ipywidgets import Layout
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label

class LogWidget(Textarea):
    log_no = 0

    def __init__(self, *args, **kwargs):
        super(Textarea, self).__init__(*args, **kwargs)

        """
        set default description and style
        """
        self.description = 'log'
        self.style = {'description_width': 'initial'}
        self.maxline = 0 # no limit
        self.layout = Layout(width='100%')
        """
        update from arguments
        vのvalidationはいたしません。
        """
        for k, v in kwargs.items():
            if k == 'description':
                self.description = v
            elif k == 'style':
                self.style = v
            elif k == 'maxline':
                self.maxline = v
            elif k == 'layout':
                self.layout = v


    def __call__(self, msg):
        """
        write log
        """
        self.log_no = self.log_no + 1
        if self.maxline > 0:
            # 最新のメッセージ + 過去maxline行分を表示します。
            self.value = str(self.log_no) + ": " + str(msg) + "\n" + "\n".join(self.value.splitlines()[:self.maxline])
        else:
            self.value = str(self.log_no) + ": " + str(msg) + "\n" + self.value

write_log = LogWidget(maxline=100)

In [2]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
import re
import ipywidgets
from utils import preprocess
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
from jupyter_clickable_image_widget import ClickableImageWidget
from jetcam.utils import bgr8_to_jpeg
import cv2
import torchvision.transforms as transforms
from xy_dataset2 import XYDataset
import time
import threading
import torch
import torchvision
import ipywidgets
#from ipyevents import Event
from PIL import Image, ImageDraw, ImageFont
import io
import traceback

In [3]:
IMG_WIDTH = 224
IMG_HEIGHT = 224

SLEEP = [50,100,200,300,400,500]
SKIP = [1,2,3,4,5,10,20,40,60]
# アノテーション編集モード毎にスキップ数を変更できるようにする
record_skip = 5
annotation_skip = 1

LOAD_CATEGORIES = ['xy','speed']
SAVE_CATEGORIES = ['xy','speed']

LOAD_DATASETS = ['X','Y','Z']
SAVE_DATASETS = []

LOAD_TASK = ['camera','dataset','interactive']
SAVE_TASK = ['dataset']

check_flag = False
running = False
sleep_time = 10
current_path = os.getcwd()
xy_filenames = []
speed_imagenames = []

def get_dirs(path):
    files = os.listdir(path)
    dirs = [f for f in files if os.path.isdir(os.path.join(path, f))]
    dirs = [f for f in files if f != ".ipynb_checkpoints"]
    dirs = sorted(dirs)
    
    return dirs

try:  
    path = os.path.join(current_path,"dataset")
    dirs = get_dirs(path)
    if not dirs:
        SAVE_DATASETS = ['dataset1']
    else:
        SAVE_DATASETS = dirs
        
except:
    SAVE_DATASETS = ['dataset1']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in SAVE_DATASETS:
    for task in SAVE_TASK:
        datasets[name] = XYDataset(task + '/' + name, SAVE_CATEGORIES, TRANSFORMS, random_hflip=True)
        
dataset = datasets[SAVE_DATASETS[0]]

In [4]:
sleep_dropdown = ipywidgets.Dropdown(options=SLEEP, description='sleep(ms)', index=0)
skip_dropdown = ipywidgets.Dropdown(options=SKIP, description='skip(枚)', index=4)
skip_movie_dropdown = ipywidgets.Dropdown(options=SKIP, description='skip(枚)', index=1)

picture_widget = ClickableImageWidget(width=224, height=224)
# 画像のバイトデータを取得
image = Image.new('RGB', (224, 224), 'white')
image_byte_arr = io.BytesIO()
image.save(image_byte_arr, format='JPEG')
# 画像を表示するウィジェットを作成
"""picture_widget = ipywidgets.Image(
    value=image_byte_arr.getvalue(),
    format='jpg',
    width=224,
    height=224
)
"""

no_widget = ipywidgets.IntText(description='no')
x_widget = ipywidgets.IntText(description='data x')
y_widget = ipywidgets.IntText(description='data y')
speed_widget = ipywidgets.IntText(description='data speed')
ai_x_widget = ipywidgets.IntText(description='AI　x')
ai_y_widget = ipywidgets.IntText(description='AI　y')
ai_speed_widget = ipywidgets.IntText(description='AI speed')
model_widget = ipywidgets.Text(description='model')
model_widget.value = "model.pth"
load_model_button = ipywidgets.Button(description='load model')
speed_slider = ipywidgets.IntSlider(description='speed', min=0, max=224, step=1, value=0, orientation='vertical')
add_speed_button = ipywidgets.Button(description='速度追加')

In [5]:
from packaging import version

torchvision_version = version.parse(torchvision.__version__)

device = torch.device('cuda')
output_dim = 2*len(LOAD_CATEGORIES)  # LOAD_CATEGORIESは事前に定義されている必要があります

if torchvision_version >= version.parse("0.13"):
    from torchvision.models.resnet import ResNet18_Weights, resnet18

    default_weights = torchvision.models.ResNet18_Weights.DEFAULT
    model = torchvision.models.resnet18(weights=default_weights)
    model.fc = torch.nn.Linear(model.fc.in_features, output_dim)
else:
    model = torchvision.models.resnet18(pretrained=True)
    model.fc = torch.nn.Linear(512, output_dim)

model = model.to(device)

In [6]:
def load_model(c):
    global torchvision_version, load_model_widget, model, device, output_dim
    
    model_name = load_model_widget.value
    device = torch.device('cuda')
    
    if torchvision_version >= version.parse("0.13"):
        # torchvision 0.13以降の場合
        from torchvision.models.resnet import ResNet18_Weights, resnet18
        
        if model_name == "[new]":
            # 新しい重みを使ってモデルをロード
            default_weights = torchvision.models.ResNet18_Weights.DEFAULT
            model = torchvision.models.resnet18(weights=default_weights)
            model.fc = torch.nn.Linear(model.fc.in_features, output_dim)
            model = model.to(device)
            write_log("[new]が選択されたのでresnet18の最新の重みから始めます(torchvision 0.13以降)。")
        else:
            model = torchvision.models.resnet18(weights=None)  # pretrained=Falseの代わり
            model.fc = torch.nn.Linear(model.fc.in_features, output_dim)
            model = model.to(device)
            model.load_state_dict(torch.load(model_name))
            write_log(model_name + "のモデルを読込ました(torchvision 0.13以降)。")

    else:
        # torchvision 0.13より前の場合
        if model_name == "[new]":
            model = torchvision.models.resnet18(pretrained=True)
            model.fc = torch.nn.Linear(512, output_dim)
            model = model.to(device)
            write_log("[new]が選択されたのでresnet18のpretrainedから始めます。")
        else:
            model = torchvision.models.resnet18(pretrained=False)
            model.fc = torch.nn.Linear(512, output_dim)
            model = model.to(device)
            model.load_state_dict(torch.load(model_name))
            write_log(model_name + "のモデルを読込ました。")
    model.eval()
    get_jetson_nano_memory_usage()

def save_model(c):
    global save_model_name_widget, model, device
    path = "./model/"
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    torch.save(model.state_dict(), path + save_model_name_widget.value)
    write_log(path + save_model_name_widget.value + "に保存しました。")

In [7]:
import time

BATCH_SIZE = 8

epochs_widget = ipywidgets.IntText(description='epochs', value=1)
eval_button = ipywidgets.Button(description='evaluate')
train_button = ipywidgets.Button(description='train')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='progress')

best_loss = float('inf')  

def train_eval(is_training):
    global BATCH_SIZE, model, dataset, optimizer, best_loss, current_path
    
    optimizer = torch.optim.Adam(model.parameters())
    
    xy_path = os.path.join(current_path,save_task_widget.value,save_datasets_widget.value,"xy")
    speed_path = os.path.join(current_path,save_task_widget.value,save_datasets_widget.value,"speed")

    xy_is_dir = os.path.isdir(xy_path)
    speed_is_dir = os.path.isdir(speed_path)
    
    xy_file_count = 0
    speed_file_count = 0
    
    if xy_is_dir:
        xy_file_count = sum(os.path.isfile(os.path.join(xy_path,name)) for name in os.listdir(xy_path))
    if speed_is_dir:
        speed_file_count = sum(os.path.isfile(os.path.join(speed_path,name)) for name in os.listdir(speed_path))
    write_log("-------------------------")
    write_log("学習を開始します。")
    write_log("データセット: " + save_task_widget.value + '/' + save_datasets_widget.value)
    write_log("XYデータ数: " + str(xy_file_count) + " Speedデータ数: " + str(speed_file_count))
    write_log("-------------------------")    
    dataset = XYDataset(save_task_widget.value + '/' + save_datasets_widget.value, SAVE_CATEGORIES, TRANSFORMS, random_hflip=True)
    train_button.disabled = True
    eval_button.disabled = True
        
    # 有効なデータのインデックスを確認
    valid_indices = []
    for i in range(len(dataset)):
        try:
            item = dataset[i]
            if item is not None and item[0] is not None:
                valid_indices.append(i)
        except Exception as e:
            err_info = traceback.format_exc()
            write_log(f"{err_info}")
            pass
        
    # 有効なインデックスを持つサブセットを作成
    valid_dataset = torch.utils.data.Subset(dataset, valid_indices)
    
    
    try:
        train_loader = torch.utils.data.DataLoader(
            valid_dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()
        epoch_count = 0
        
        while epochs_widget.value > 0:
            epoch_start_time = time.time()  # エポック開始時間を記録
            epoch_count += 1
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                if images is None or xy is None:
                    print("Warning: None type data found at index", i)
                    continue
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
            
            # エポック終了時に時間を記録してログに出力
            epoch_end_time = time.time()
            epoch_duration = epoch_end_time - epoch_start_time
            write_log(f"{epoch_count} Epoch目: {epoch_duration:.2f}秒")
            #get_jetson_nano_memory_usage()
            
            # 最小損失をチェックし、必要に応じてモデルを保存
            if is_training and loss_widget.value < best_loss:
                best_loss = loss_widget.value
                model_dir = './model'
                if not os.path.exists(model_dir):
                    os.makedirs(model_dir)
                torch.save(model.state_dict(), model_dir + "/" + 'best_model.pth')
                write_log(f"新しいベストモデルが保存されました。Epoch loss: {best_loss:.4f}")

            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except Exception as e:
        train_button.disabled = False
        eval_button.disabled = False
        err_info = traceback.format_exc()
        write_log(f"{err_info}")
        pass
    
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False

train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

In [8]:
from os.path import join
import subprocess
import datetime
import glob

def extract_numbers(filename):
    matches = re.findall(r'(\d+)', filename)
    if matches and len(matches) >= 3: 
        return int(matches[-1])  
    else:
        return float('inf') 

def get_file_names(path):
    if not os.path.exists(path):
        return []  # pathが存在しない場合は空のリストを返す
    file_names = os.listdir(path)
    file_names = [os.path.join(path, file_name) for file_name in file_names]
    image_names = []

    image_names = sorted(file_names, key=lambda f: extract_numbers(os.path.basename(f)))
    image_names = [f for f in image_names if os.path.splitext(f)[1].lower() == ".jpg"]
    
    return image_names

def bad_get_file_names(path):
    # 連続的に書いても、書き込み時間がずれてしまうので、これは使えない。
    file_paths = glob.glob(os.path.join(path, '*.jpg'))  # jpgファイルのみを取得
    # ファイルの最終変更時刻に基づいてソート
    sorted_file_paths = sorted(file_paths, key=os.path.getmtime)
    return sorted_file_paths


def load_img(no):
    global running, img, play_num

    image_path = None
    img = None
    x = 0
    y = 0
    speed = 0
    speed_widget.value = speed
    speed_slider.value = speed

    flag_annotation_xy = False
    flag_annotation_speed = False
    name = save_datasets_widget.value
    dir1 = load_task_widget.value
    dir2 = load_datasets_widget.value
    dir3 = LOAD_CATEGORIES[0]
    if name in datasets:
        dataset = datasets[name]
    
        if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
            annotation = dataset.find_annotation(f"{dir1}_{dir2}_{dir3}_{no}")
        elif annotation_target == ANNOTATION_TARGET_ANNOTATION_DATA:
            annotation = dataset.find_annotation_from_index(no)

        if annotation:
            image_path = annotation['image_path']
            if image_path is not None:
                img = cv2.imread(image_path)
            if img is not None:
                if annotation['category'] == 'xy':
                    # 画像にマーキングを追加し、ウィジェットに表示するなどの追加処理
                    x, y = annotation['x'], annotation['y']
                    x_widget.value = x
                    y_widget.value = y
                    flag_annotation_xy = True
                elif annotation['category'] == 'speed':
                    speed = annotation['y']
                    speed_widget.value = speed
                    speed_slider.value = speed
                    flag_annotation_speed = True
            else:
                write_log("画像を読み込めませんでした。")
                running = False
                return
        else:
            #write_log(f"{no}は未アノテーション画像です。")
            pass
    else:
        if name is None:
            write_log("データセットがありません。")
        else:
            write_log(f"データセット {name} が見つかりません。")
        running = False

    if not flag_annotation_xy:
        if len(xy_imagenames) > 0:
            if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
                image_path, x, y = load_img_xy_from_storage(no)
    if not flag_annotation_speed:
        if len(speed_imagenames) > 0:
            if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
                _, speed = load_img_speed_from_storage(no)
                if speed is not None:
                    speed_widget.value = speed
                    speed_slider.value = speed
                if image_path is None:
                    image_path = _

    if image_path is None:
        write_log(f"画像が見つかりません。")
        running = False
        return

    if img is None:
        img = cv2.imread(image_path)
    if img is None:
        write_log(f"{image_path} の読み込みに失敗しました。")
        running = False
        return

    marked_img = img.copy()
    
    black_color = (0, 0, 0)
    blue_color = (255, 0, 0)
    green_color = (0, 255, 0)

    if flag_annotation_xy:
        marked_img = cv2.circle(marked_img, (int(x), int(y)), 8, green_color, 3)
    
    try:
        preprocessed = preprocess(img)
        output = model(preprocessed).detach().cpu().numpy().flatten()
        result_x = output[0]
        result_y = output[1]
        result_speed = output[3]
        result_x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
        result_y = int(IMG_HEIGHT * (result_y / 2.0 + 0.5))
        result_speed = int(IMG_HEIGHT * (result_speed / 2.0 + 0.5))
        
        ai_x_widget.value = result_x
        ai_y_widget.value = result_y
        ai_speed_widget.value = result_speed
        
        marked_img = cv2.circle(marked_img, (int(result_x), int(result_y)), 8, blue_color, 3)
        if flag_annotation_xy:
            marked_img = cv2.circle(marked_img, (int(x), int(y)), 8, green_color, 3)
        marked_img = cv2.line(marked_img,(112,0),(112,224),(255,255,0),1)
        marked_img = cv2.line(marked_img,(82,0),(82,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(52,0),(52,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(22,0),(22,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(142,0),(142,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(172,0),(172,224),(100,100,255),1)
        marked_img = cv2.line(marked_img,(202,0),(202,224),(100,100,255),1)

        marked_img = cv2.line(marked_img,(0,112),(224,112),(255,255,0),1)
        marked_img = cv2.line(marked_img,(0,82),(224,82),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,52),(224,52),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,22),(224,22),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,142),(224,142),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,172),(224,172),(100,100,255),1)
        marked_img = cv2.line(marked_img,(0,202),(224,202),(100,100,255),1)
        
        # Speed
        if result_speed> 224:
            result_speed = 224
        elif result_speed < 0:
            result_speed = 0
            
        marked_img = cv2.line(marked_img,(218,0),(218,224),black_color,5)
        marked_img = cv2.line(marked_img,(219,224-result_speed),(219,224),blue_color,3)
        marked_img = cv2.putText(marked_img,"speed:"+str(result_speed),(160,215),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255))
    
        if int(speed) != 0:
            marked_img = cv2.line(marked_img,(1,0),(1,224),black_color,5)
            marked_img = cv2.line(marked_img,(2,224-int(speed)),(2,224),green_color,3)        
    except Exception as e:
        err_info = traceback.format_exc()
        write_log(f"{err_info}")
    
    picture_widget.value = bgr8_to_jpeg(marked_img)

    if running == True:
        play_num += 1
        if play_num % 10 == 0:
            write_log(f"{play_num} 回目の再生")
    else:
        next_image_button.disabled = False
        prev_image_button.disabled = False
        write_log(str(no) + "枚目の" + image_path + "を読込ました。") 


def load_img_xy_from_storage(no):
    global running
    if no >= len(xy_imagenames):
        no_widget.value = no - 1
        write_log("xyファイルが存在しません。" + str(len(xy_imagenames)-1) + "以内の値を設定してください。")
        running = False
        return None, None, None
    
    image_path = xy_imagenames[no]
    
    pattern = '.*/(\d+)_(\d+)_.*'
    xy_result = re.match(pattern, image_path)
    x = 0
    y = 0
    if xy_result:
        x = xy_result.group(1)
        y = xy_result.group(2)
    x_widget.value = x
    y_widget.value = y

    return image_path, x, y

def load_img_speed_from_storage(no):
    global running
    if no >= len(speed_imagenames):
        no_widget.value = no - 1
        write_log("speed ファイルが存在しません。" + str(len(speed_imagenames)-1) + "以内の値を設定してください。")
        running = False
        return None, None
        
    speed = 0
    try:
        image_path = speed_imagenames[no]
        speed_result = re.match(pattern, image_path)
        if speed_result:
            speed = int(speed_result.group(2))
            speed_widget.value = speed
            speed_slider.value = speed
    except:
        speed_widget.value = speed
        speed_slider.value = speed

    return image_path, speed

def del_pic(c):
    global no, load_task_widget,load_datasets_widget
    name = save_datasets_widget.value
    dataset = datasets[name]
    no = no_widget.value

    dir1 = load_task_widget.value
    dir2 = load_datasets_widget.value
    dir3 = LOAD_CATEGORIES[0]
    if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
        task_dataset_category_no = f"{dir1}_{dir2}_{dir3}_{no}"
    elif annotation_target == ANNOTATION_TARGET_ANNOTATION_DATA:
        task_dataset_category_no = dataset.find_task_dataset_category_no_from_index(no)
    filename = dataset.delete_entry(task_dataset_category_no)

    if filename:
        write_log(f"アノテーションから{filename}を削除しました。")
        load_img(no)
    else:
        write_log(f"アノテーションに削除したいファイルがありません。")
    
def load_dataset(c):
    global img,load_flag,current_path
    dataset_path = os.path.join(current_path,load_task_widget.value,load_datasets_widget.value)
    write_log("データセット: " + dataset_path + "を読込みます。")
    load_flag = True
    no_widget.value = 0
    no = 0
    write_log("初回の読込みには時間がかかります。(30秒〜1分)")
    load_img(no)
    get_jetson_nano_memory_usage()

def next_pic(c):
    global no,x,y,load_flag,check_flag
    load_flag = True
    check_flag = False
    no = no_widget.value
    no = int(no) + skip_dropdown.value
    no_widget.value = no
    next_image_button.disabled = True
    prev_image_button.disabled = True
    load_img(no)

    
def prev_pic(c):
    global no,x,y,load_flag,check_flag
    load_flag = True
    check_flag = False
    no = no_widget.value
    no = int(no) - skip_dropdown.value
    if no < 0:
        no = 0
    no_widget.value = no
    next_image_button.disabled = True
    prev_image_button.disabled = True
    load_img(no)

def file_count():
    global current_path
    
    try:
        xy_path = os.path.join(current_path,save_task_widget.value,save_datasets_widget.value,"xy")
        speed_path = os.path.join(current_path,save_task_widget.value,save_datasets_widget.value,"speed")
        
        xy_is_dir = os.path.isdir(xy_path)
        
        if xy_is_dir:
            xy_file_count = sum(os.path.isfile(os.path.join(xy_path,name)) for name in os.listdir(xy_path))
            datasets_xy_count_widget.value = xy_file_count
        else:
            datasets_xy_count_widget.value = 0
        
        speed_is_dir = os.path.isdir(speed_path)
        
        if speed_is_dir:
            speed_file_count = sum(os.path.isfile(os.path.join(speed_path,name)) for name in os.listdir(speed_path))
            datasets_speed_count_widget.value = speed_file_count
        else:
            datasets_speed_count_widget.value = 0
            
    except Exception as e:
        #err_info = traceback.format_exc()
        #write_log(f"{err_info}")
        datasets_xy_count_widget.value = 0
        datasets_speed_count_widget.value = 0

def save_snapshot(_, content, msg):
    global img,x,y,load_flag,save_datasets_widget,save_task_widget,save_category_widget, check_flag
    
    if content['event'] == 'click' and load_flag == True:
        load_flag = False
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        write_log("x" + str(x))
        write_log("y" + str(y))
        
    if img is None:
        write_log(f'アノテーション可能な画像がありません。')
        return
    remarked_img = img.copy()
    remarked_img = cv2.circle(remarked_img, (int(x), int(y)), 8, (0, 255, 0), 3)
    
    picture_widget.value = bgr8_to_jpeg(remarked_img)
    name = save_datasets_widget.value
    if name is None or name == "":
        write_log("保存先のデータセット名を作成してください。")
        return
    else:
        try:
            dataset = datasets[name]
            no = no_widget.value
            dir1 = load_task_widget.value
            dir2 = load_datasets_widget.value
            dir3 = LOAD_CATEGORIES[0]
            if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
                task_dataset_category_no = f"{dir1}_{dir2}_{dir3}_{no}"
            elif annotation_target == ANNOTATION_TARGET_ANNOTATION_DATA:
                task_dataset_category_no = dataset.find_task_dataset_category_no_from_index(no)
                if task_dataset_category_no is None:
                    write_log(f"アノテーションに編集したいファイルがありません。")
                    return
            filename = dataset.save_entry("xy", img, x, y, task_dataset_category_no)
            write_log(f"{filename} に保存しました。")
            file_count()
        except:
            err_info = traceback.format_exc()
            write_log(f"{err_info}")

    no = no_widget.value
    if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
        task_dataset_category_no = f"{dir1}_{dir2}_{dir3}_{no}"
        annotation = dataset.find_annotation(task_dataset_category_no)
    elif annotation_target == ANNOTATION_TARGET_ANNOTATION_DATA:
        annotation = dataset.find_annotation_from_index(no)

    write_log(f"annotation: {annotation}")

    if annotation_action == ANNOTATION_ACTION_DEFAULT:
        load_img(no)
    elif annotation_action == ANNOTATION_ACTION_NEXT_PIC:
        time.sleep(0.5)
        next_pic(None)
        
def save_speed(c):
    global img,speed_slider,save_datasets_widget,save_task_widget
    speed = speed_slider.value
    remarked_img = img.copy()
    name = save_datasets_widget.value
    dataset = datasets[name]
    no = no_widget.value
    dir1 = load_task_widget.value
    dir2 = load_datasets_widget.value
    dir3 = LOAD_CATEGORIES[1]
    if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
        task_dataset_category_no = f"{dir1}_{dir2}_{dir3}_{no}"
    elif annotation_target == ANNOTATION_TARGET_ANNOTATION_DATA:
        task_dataset_category_no = dataset.find_task_dataset_category_no_from_index(no)
        if task_dataset_category_no is None:
            write_log(f"アノテーションに編集したいファイルがありません。")
            return
    filename = dataset.save_entry("speed", img, 0, speed, task_dataset_category_no)
    write_log(f"{filename} に保存しました。")
    file_count()
    
def live():
    global no,running, play_num
    load_flag = True
    play_num = 0
    no = no_widget.value
    while running:
        no += skip_dropdown.value
        no_widget.value = no
        try:
            load_img(no)
        except:
            write_log("no: " + no + "のファイルの読込に失敗")
        time.sleep(sleep_time/1000)
    
def play(c):
    global running, execute_thread, sleep_time, check_flag
    sleep_time = sleep_dropdown.value
    running = True
    check_flag = False
    execute_thread = threading.Thread(target=live)
    execute_thread.start()
    
def stop(c):
    global running, execute_thread, load_flag, check_flag
    running = False
    load_flag = True
    check_flag = False
    try:
        execute_thread.join()
        write_log("STOP")
    except:
        write_log("現在再生されていません。")

def create_dataset(c):
    global datasets_name_widget, save_datasets_widget
    new_dataset_name = datasets_name_widget.value
    
    if new_dataset_name not in SAVE_DATASETS:
        SAVE_DATASETS.append(new_dataset_name)

        save_datasets_widget.options = SAVE_DATASETS
        save_datasets_widget.value = new_dataset_name
    
    datasets = {}
    for name in SAVE_DATASETS:
        for task in SAVE_TASK:
            datasets[name] = XYDataset(task + '/' + name, SAVE_CATEGORIES, TRANSFORMS, random_hflip=True)
            
    dataset = datasets[new_dataset_name]  # 新しいデータセットを指定
    write_log("Datasetを作成しました：" + new_dataset_name)


picture_widget.on_msg(save_snapshot)

# Eventインスタンスを作成
#image_event_handler = Event(source=picture_widget, watched_events=['click'])
# イベントハンドラを設定
#image_event_handler.on_dom_event(image_handle_event)


# 画像の操作
play_button = ipywidgets.Button(description='▶')
stop_button = ipywidgets.Button(description='⏹')

# アノテーション対象を選択可能にする
ANNOTATION_TARGET_RECORD_DATA = 0
ANNOTATION_TARGET_ANNOTATION_DATA = 1
annotation_target = ANNOTATION_TARGET_RECORD_DATA
annotation_target_selector = ipywidgets.widgets.RadioButtons(
    options = [('レコードデータ（追加 + 修正)', ANNOTATION_TARGET_RECORD_DATA), ('アノテーションデータ (修正のみ)', ANNOTATION_TARGET_ANNOTATION_DATA)],
    description = 'アノテーション対象を選択:',
    value = annotation_target,
    disabled = False
)
def annotation_target_selector_handler(change):
    global annotation_target
    global running
    if change['type'] == 'change' and change['name'] == 'value':
        annotation_target = change['new']
        no = 0
        no_widget.value = no
        if running:
            running = False
            time.sleep(0.5)
        if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
            skip_dropdown.value = record_skip
        elif annotation_target == ANNOTATION_TARGET_ANNOTATION_DATA:
            skip_dropdown.value = annotation_skip
        load_img(no)
annotation_target_selector.observe(annotation_target_selector_handler, names='value')

# アノテーション追加後にnext_picを呼ぶかどうか
ANNOTATION_ACTION_DEFAULT = 0
ANNOTATION_ACTION_NEXT_PIC = 1
annotation_action = ANNOTATION_ACTION_DEFAULT
annotation_action_selector = ipywidgets.widgets.RadioButtons(
    options = [('デフォルト', ANNOTATION_ACTION_DEFAULT), ('next_pic', ANNOTATION_ACTION_NEXT_PIC)],
    description = '追加後の動作選択:',
    value = annotation_action,
    disabled = False
)
def annotation_action_selector_handler(change):
    global annotation_action
    if change['type'] == 'change' and change['name'] == 'value':
        annotation_action = change['new']


annotation_action_selector.observe(annotation_action_selector_handler, names='value')

next_image_button = ipywidgets.Button(description='>')
prev_image_button = ipywidgets.Button(description='<')
load_image_button = ipywidgets.Button(description='読込')
update_image_button = ipywidgets.Button(description='更新')
delete_image_button = ipywidgets.Button(description='削除')

save_model_button = ipywidgets.Button(description='save model')

load_model_widget = ipywidgets.Dropdown(options=[],description='読込モデル')
load_model_time_widget = ipywidgets.Text(description='作成日時')
save_model_name_widget = ipywidgets.Text(description='保存モデル名',value="model.pth")

dataset_create_button = ipywidgets.Button(description='Create dataset')
datasets_name_widget = ipywidgets.Text(description='Name')

dataset_create_button.on_click(create_dataset)

play_button.on_click(play)
stop_button.on_click(stop)

add_speed_button.on_click(save_speed)

load_model_button.on_click(load_model)
save_model_button.on_click(save_model)
load_image_button.on_click(load_dataset)
next_image_button.on_click(next_pic)
prev_image_button.on_click(prev_pic)
delete_image_button.on_click(del_pic)

load_datasets_widget = ipywidgets.Dropdown(options=LOAD_DATASETS, description='dataset', index=0)
save_datasets_widget = ipywidgets.Dropdown(options=SAVE_DATASETS, description='dataset')
datasets_xy_count_widget = ipywidgets.IntText(description='XYデータ数')
datasets_speed_count_widget = ipywidgets.IntText(description='速度データ数')

def set_dataset(change):
    global dataset
    datasets[change['new']] = XYDataset(save_task_widget.value + '/' + change['new'], SAVE_CATEGORIES, TRANSFORMS, random_hflip=True)
    #dataset = datasets[change['new']]
    #write_log(change['new'])
save_datasets_widget.observe(set_dataset, names='value')

load_task_widget = ipywidgets.Dropdown(options=LOAD_TASK, description='task')
save_task_widget = ipywidgets.Dropdown(options=SAVE_TASK,  value=SAVE_TASK[0], description='task')

def change_load_task(change):
    global dataset, current_path, load_task_widget, load_datasets_widget
    try:
        path = os.path.join(current_path,load_task_widget.value)
        dirs = get_dirs(path)
        load_datasets_widget.options = dirs
        if len(dirs) > 0:
            load_datasets_widget.value = dirs[0]
    except Exception:
        err_info = traceback.format_exc()
        write_log(f"{err_info}")
        write_log(path + "が存在していません。")
        load_datasets_widget.options = []
load_task_widget.observe(change_load_task, names='value')
change_load_task(LOAD_TASK[0])

def change_load_datasets(change):
    if load_datasets_widget.value:
        write_log("change_load_datasets")
        update_xy_imagenames()

load_datasets_widget.observe(change_load_datasets, names='value')

def change_save_task(change):
    global dataset, current_path
    try:
        path = os.path.join(current_path,save_task_widget.value)
        if not os.path.exists(path):
            subprocess.call(['mkdir', '-p', path])
        dirs = get_dirs(path)
        save_datasets_widget.options = dirs
    except:
        write_log(path + "が存在していません。")
        save_datasets_widget.options = ['']
save_task_widget.observe(change_save_task, names='value')
change_save_task(SAVE_TASK[0])

def change_save_dataset(change):
    global dataset, current_path
    file_count()
save_datasets_widget.observe(change_save_dataset, names='value')
change_save_dataset(SAVE_DATASETS[0])

def change_sleep(change):
    global sleep_time, sleep_dropdown
    sleep_time = sleep_dropdown.value
sleep_dropdown.observe(change_sleep, names='value')

def change_skip(change):
    global record_skip, annotation_skip
    if annotation_target == ANNOTATION_TARGET_RECORD_DATA:
        record_skip = skip_dropdown.value
    elif annotation_target == ANNOTATION_TARGET_ANNOTATION_DATA:
        annotation_skip = skip_dropdown.value

skip_dropdown.observe(change_skip, names='value')

def model_list(change):
    global load_model_widget
    try:
        files = glob.glob('./model/*.pth', recursive=True)
        files.insert(0,"[new]")
        load_model_widget.options = files
        load_model_time_widget.value = ""
    except:
        load_model_widget.options = []
model_list("list")

def change_file(change):
    global load_model_widget
    try:
        file = load_model_widget.value
        ts = os.path.getctime(file)
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        load_model_time_widget.value = s
    except:
        load_model_time_widget.value = ""
load_model_widget.observe(change_file, names='value')

def update_image(change):
    global load_flag, no
    load_flag = True
    no = no_widget.value
    load_img(no)

update_image_button.on_click(update_image)

load_category_widget = ipywidgets.Dropdown(options=LOAD_CATEGORIES, description='category')
save_category_widget = ipywidgets.Dropdown(options=SAVE_CATEGORIES, description='category')

def update_xy_imagenames():
    global xy_imagenames, speed_imagenames
    load_task_value = load_task_widget.value
    load_category_value = load_category_widget.value
    load_datasets_value = load_datasets_widget.value
    if load_task_value and load_category_value and load_datasets_value:
        # xy_pathを構築
        xy_path = os.path.join(current_path, load_task_value, load_datasets_value, "xy")
        # xy_path内のファイル名のリストを取得
        xy_imagenames = get_file_names(xy_path)
        
        # speed_pathを構築
        speed_path = os.path.join(current_path, load_task_value, load_datasets_value, "speed")
        # speed_path内のファイル名のリストを取得
        speed_imagenames = get_file_names(speed_path)

update_xy_imagenames()

In [9]:
import numpy as np
from functools import partial

WIDTH = 80
HEIGHT = 80
SIZE = 8

check_image_button = ipywidgets.Button(description=f'{SIZE}個単位チェック')
check_next_images_button = ipywidgets.Button(description=f'[{SIZE}]>')
check_prev_images_button = ipywidgets.Button(description=f'<[-{SIZE}]')
check_start_index_widget = ipywidgets.IntText(description='開始位置')
check_end_index_widget = ipywidgets.IntText(description='終了位置')
check_image_count_widget = ipywidgets.IntText(description='最終画像位置')
check_update_button = ipywidgets.Button(description='更新')

# 画像を表示するウィジェット
snapshot_widgets = []
snapshot_button_widgets = []


def edit_image(index, b):
    global load_flag,no,check_flag
    no = check_no + index
    load_flag = True
    check_flag = False
    load_img(no)
    no_widget.value = no
    
for i in range(SIZE):
    image = ipywidgets.Image(width=WIDTH, height=HEIGHT)
    edit_button = ipywidgets.Button(description="編集", layout=ipywidgets.Layout(width=f'{WIDTH}px', height=f'30px'))
    edit_button.on_click(partial(edit_image, i))
    black_image = np.zeros((HEIGHT, WIDTH, 3), dtype=np.uint8)
    image.value = bgr8_to_jpeg(black_image)
    snapshot_widgets.append(image)
    snapshot_button_widgets.append(VBox([edit_button,image]))

def get_load_dataset_length():
    check_image_count_widget.value = len(xy_filenames)
    last_no = len(xy_filenames)
    return last_no

def next_images(c):
    global check_no,last_no,check_flag
    check_next_images_button.disabled = True
    check_prev_images_button.disabled = True
    write_log(f"check_flag:"+str(check_flag))
    if check_flag == False:
        check_no = no_widget.value
        check_flag = True
    else:
        check_no += SIZE
        write_log(f"check_no: {check_no}")
    try:
        last_no = get_load_dataset_length()
        if check_no < last_no:
            load_img(check_no)
        else:
            check_next_images_button.disabled = False
            check_prev_images_button.disabled = False
    except:
        check_next_images_button.disabled = False
        check_prev_images_button.disabled = False
    
def prev_images(c):
    global check_no,check_flag
    check_next_images_button.disabled = True
    check_prev_images_button.disabled = True
    write_log(f"check_flag:"+str(check_flag))
    if check_flag == False:
        check_no = no_widget.value
        check_no -= SIZE
        check_flag = True
    else:
        check_no -= SIZE
        write_log(f"check_no: {check_no}")
    try:
        last_no = get_load_dataset_length()
        if check_no < 0:
            check_no = 0
        load_img(check_no)
    except:
        check_next_images_button.disabled = False
        check_prev_images_button.disabled = False

def update_images(c):
    global check_no
    check_no = check_start_index_widget.value
    load_img(check_no)
    
check_no = 0
check_image_button.on_click(load_img)
check_prev_images_button.on_click(prev_images)
check_next_images_button.on_click(next_images)
check_update_button.on_click(update_images)

In [10]:
movie_button = ipywidgets.Button(description='動画の作成')
movie_name_widget = ipywidgets.Text(description='動画名',value="run_video")

def make_movie(change):
    global model,current_path
    
    if not movie_name_widget.value.strip():
        write_log("ファイル名を指定してください。")
        return 
    write_log("動画を作成します。")
    path = os.path.join(current_path, "video/")
    if not os.path.exists(path):
        subprocess.call(['mkdir', '-p', path])
    output = path + movie_name_widget.value + ".mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(30 / skip_movie_dropdown.value)
    outfh = cv2.VideoWriter(output, fourcc, fps, (224, 224))
    file_list = sorted(
        glob.glob(load_task_widget.value + '/' + load_datasets_widget.value + '/xy/*.jpg'),
        key=os.path.getmtime
    )
    
    xy_path = os.path.join(current_path, load_task_widget.value, load_datasets_widget.value, "xy")     
    file_list = os.listdir(xy_path)
    file_list = [os.path.join(xy_path, file_name) for file_name in file_list if file_name.endswith('.jpg')]    
    file_list = sorted(file_list, key=lambda f: extract_numbers(os.path.basename(f)))
    
    
    try:
        res_num = len(file_list)
        
        count = 0
        skip_movie = skip_movie_dropdown.value
        terminal_time = 1/(30/skip_movie)
        current_time = 0
        process_time = 0
        total_process_time = 0
        for i, file_name in enumerate(file_list):
            
            if i % skip_movie == 0:
                current_time += terminal_time
                img = cv2.imread(file_name)
                
                process_time = time.time()
                preprocessed = preprocess(img)
                output = model(preprocessed).detach().cpu().numpy().flatten()
                result_x = float(output[0])
                result_y = float(output[1])
                result_x = int(IMG_WIDTH * (result_x / 2.0 + 0.5))
                result_y = int(IMG_HEIGHT * (result_y / 2.0 + 0.5))    
                img = cv2.circle(img, (int(result_x), int(result_y)), 8, (255, 0, 0), 3)

                # Speed
                result_speed = output[3]
                result_speed = int(IMG_WIDTH * (result_speed / 2.0 + 0.5))
                if result_speed > 224:
                    result_speed = 244
                elif result_speed < 0:
                    result_speed = 0
                img = cv2.line(img,(218,0),(218,224),(0,0,0),5)
                img = cv2.line(img,(219,224-result_speed),(219,224),(0,140,255),3)
                img = cv2.putText(img,"speed:"+str(result_speed),(160,215),cv2.FONT_HERSHEY_SIMPLEX,0.3,(255,255,255))
                total_process_time += time.time() - process_time 
                
                if i % (skip_movie*10) == 0:
                    write_log(f"{current_time:.1f}秒まで完了, 推論処理平均: {total_process_time/10*1000:.1f}ms, {int(i/skip_movie)}枚目/{int(res_num/skip_movie)}枚中を処理中")
                    total_process_time = 0
                outfh.write(img)
                del img
    finally:
        # エラーが発生しても確実にリソースを解放する
        outfh.release()
        write_log("動画の出力が完了しました。")
        get_jetson_nano_memory_usage()

movie_button.on_click(make_movie)

In [11]:
import subprocess
import re

used_memory_widget = ipywidgets.IntText(description='Useメモリ', value=1)
total_memory_widget = ipywidgets.IntText(description='全メモリ', value=1)
memory_button = ipywidgets.Button(description='使用メモリ量の取得')

def get_jetson_nano_memory_usage(event=None):
    command = 'tegrastats'
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        
        mem_usage_pattern = re.compile(r'RAM (\d+)/(\d+)MB')
        
        max_lines_to_read = 10
        for _ in range(max_lines_to_read):
            line = process.stdout.readline()
            if not line:
                break 
            matches = mem_usage_pattern.search(line)
            if matches:
                used_memory_widget.value = int(matches.group(1))
                total_memory_widget.value = int(matches.group(2))
                write_log("使用メモリ： " + str(used_memory_widget.value) + "/" + str(total_memory_widget.value))
                process.kill()
                return
        
        process.kill()  
        return

    except subprocess.CalledProcessError as e:
        return

get_jetson_nano_memory_usage()
memory_button.on_click(get_jetson_nano_memory_usage)

In [12]:
separator = ipywidgets.HTML('<hr style="border-color:gray;margin:10px 0"/>')
title1 = ipywidgets.HTML('<b>【1.使用する推論モデル】</b> [New]は新規モデル。')
title2 = ipywidgets.HTML('<b>【2.読込元データセット】</b> アノテーションを実施するデータセットを選択。')
title3 = ipywidgets.HTML('<b>【3.保存先データセット】</b> データセットの保存先を選択。')
title4 = ipywidgets.HTML('<b>【4.アノテーションの実施】</b> 緑◯がアノテーション, 青◯がAIでの推論。車両の走らせたい場所で、画面をクリックすると保存先データセットのxyにデータが登録されます。Speedは[速度追加]で追加します。')
title5 = ipywidgets.HTML('<b>【5.学習】</b> EPOCH指定で学習できます。')
title6 = ipywidgets.HTML('<b>【6.評価動画の作成】</b> 動画を作成します。')

data_collection_widget = ipywidgets.VBox([
    separator,
    title1,
    ipywidgets.HBox([load_model_widget,load_model_time_widget,load_model_button]),
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    write_log,
    separator,
    title2,
    ipywidgets.HBox([load_datasets_widget,load_task_widget,load_image_button]),
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    write_log,
    separator,
    title3,
    ipywidgets.HBox([save_datasets_widget,save_task_widget]),
    ipywidgets.HBox([datasets_xy_count_widget,datasets_speed_count_widget]),
    ipywidgets.HBox([Label('datasetの新規作成'),datasets_name_widget,dataset_create_button]),
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    write_log,
    separator,
    title4,
    ipywidgets.HBox([no_widget,update_image_button,delete_image_button]), 
    skip_dropdown,
    ipywidgets.HBox([picture_widget,
                     ipywidgets.VBox([speed_slider,add_speed_button]),
                     ipywidgets.VBox([
                         ipywidgets.HBox([annotation_target_selector, annotation_action_selector]),
                         ipywidgets.HBox([play_button,stop_button]),
                         ipywidgets.HBox([prev_image_button,next_image_button])
                     ])
    ]),
    #ipywidgets.HBox(snapshot_button_widgets),
    ipywidgets.HBox([save_datasets_widget,save_task_widget]),
    ipywidgets.HBox([datasets_xy_count_widget,datasets_speed_count_widget]),
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    write_log,
    separator,
    title5,
    ipywidgets.HBox([save_datasets_widget,save_task_widget]),
    ipywidgets.HBox([datasets_xy_count_widget,datasets_speed_count_widget]),
    ipywidgets.HBox([epochs_widget,train_button,eval_button]),
    ipywidgets.HBox([progress_widget,loss_widget]),
    ipywidgets.HBox([save_model_name_widget,save_model_button]),
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    write_log,
    separator,
    title6,
    ipywidgets.HBox([load_datasets_widget,load_task_widget]),
    ipywidgets.HBox([movie_name_widget,skip_movie_dropdown,movie_button]),
    ipywidgets.HBox([used_memory_widget,total_memory_widget,memory_button]),
    write_log,
])
display(data_collection_widget)